# 查看A股均线多头的情况

## 首先导入相关的库

In [24]:
import sys
import os
sys.path.append("../../FinanceDataSource")
import FinanceDataSource
import tushare as ts
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns #要注意的是一旦导入了seaborn，matplotlib的默认作图风格就会被覆盖成seaborn的格式
import pandas as pd
import numpy as np
from pandas import DataFrame
from FinanceDataSource import get_cn_stocks
from FinanceDataSource import get_data
from FinanceDataSource import str_cn_stock
from FinanceDataSource import get_cn_bars_all
import talib
import datetime

## 保存均线多头信息的类。

In [25]:
class Ma_duotou():
    """
    这个类仅仅是保存多头信息的。我打算保存如下信息
    Attributes	:
        @book_id : 股票id
        @DateFrame : 将股票这几天的信息全部切片保存起来。方便以后运算。
    functions	:

    """

    def __init__(self, book_id, df, start_date):
        """
            Description :
            Arg :
            Returns :
            Raises	 :
        """
        self.book_id = book_id
        self.df_duotou = df
        self.start_date = start_date

## 如下几个是取得均线多头的相关信息的。

In [28]:
def get_ma_duotou_indicator(book_id, lst_ma):
    """
        Description : 将一个股票的均线多头用逻辑数组标记出来，并保存起来。
        Arg :
        Returns :
        Raises	 :
    """
    _close = get_cn_bars_all(book_id, fields='close')
    # 将所有的均线保存在这个数组中。
    _lst_ma = []
    for _ma in lst_ma:
        _lst_ma.append(talib.SMA(_close, _ma))
    _ma_0 = talib.SMA(_close, lst_ma[0])
    _ma_1 = talib.SMA(_close, lst_ma[1])
    _ma_2 = talib.SMA(_close, lst_ma[2])
    _ma_3 = talib.SMA(_close, lst_ma[3])

    _bool_up = list(
        map(lambda a, b, c, d:
            a > b and b > c and c > d,
            _ma_0, _ma_1, _ma_2, _ma_3))

    return _bool_up


def get_lst_ma_duotou(book_id, ma_duotou_indicator):
    """
        Description : 这个是从均线多头的逻辑数组中切片出连续多头的。
        Arg :
        Returns :
        Raises	 :
    """
    _data = get_cn_bars_all(book_id)     # 取得这个股票的k线
    _i = 0                                  # 遍历
    _lst_duotou = []                        # 保存所有的多头
    _count = len(ma_duotou_indicator)
    while _i < _count:    # 遍历
        if ma_duotou_indicator[_i]:         # 如果找到多头均线的起始点
            _start = _i                     # 保存一开始的下表
            _start_date = datetime.datetime.strptime(str(_data["datetime"][_i]),"%Y%m%d000000").strftime("%Y-%m-%d")
            while _i < _count and ma_duotou_indicator[_i]:  # 看看一共连续多少。
                _i = _i + 1
            # 创建一个类来保存这些信息吧
            # 保存的信息只是2点，一个就是股票id
            # 另外一个就是K线数据，用切片的来的。
            _cls_ma_duotou = Ma_duotou(book_id, _data[_start:_i],_start_date)
            _lst_duotou.append(_cls_ma_duotou)
        _i = _i + 1     # 递增。
    return _lst_duotou


def get_all_lst_ma_duotou(lst_ma):
    """
        Description : 取得所有股票的多头数据，可以用其他程序进行分析的。
        Arg :
        Returns :
        Raises	 :
    """
    # 要记录所有多头信息的里列表
    _lst_duotou = []
    # 取得所有的股票
    _all_cn_stock = get_cn_stocks()

    # 遍历所有的股票, 先找出所有的多头趋势来。
    for _book_id in _all_cn_stock:
        _bool_up = get_ma_duotou_indicator(_book_id, lst_ma)
        _lst_duotou.extend(get_lst_ma_duotou(_book_id, _bool_up))
    return _lst_duotou

In [29]:
%time _lst_ma_duotou = get_all_lst_ma_duotou([5, 10, 20, 60])

Wall time: 12.8 s


In [30]:
print("一共搜索到处于均线多头状态的数量：{}".format(len(_lst_ma_duotou)))

一共搜索到处于均线多头状态的数量：166322


## 接下来就是怎么看这些数据了

In [31]:
# 我想建立一个表格，用来汇总这些数据
# 列：起始时间，股票ID，持续天数，一开始的值，最后的值，以及增长比率
_start_date=[]
_book_id=[]
_continue_days=[]
_close_first=[]
_close_last=[]
_close_up_ratio=[]
# 然后我要遍历
for _ma_duotou in _lst_ma_duotou:
    _start_date.append(_ma_duotou.start_date)
    _book_id.append(_ma_duotou.book_id)
    _continue_days.append(len(_ma_duotou.df_duotou))
    _close_first.append(_ma_duotou.df_duotou['close'][0])
    _close_last.append(_ma_duotou.df_duotou['close'][-1])
    _close_up_ratio=(_close_last[-1]-_close_first[-1])/_close_first[-1]*100
    
#然后保存到pandas的DataFrame类型中，方便查看。
_dict_ma_duotou_1={
    'start_date':_start_date,
    'book_id':_book_id,
    'continue_days':_continue_days,
    'close_up_ratio':_close_up_ratio
}
_df_ma_duotou_1=pd.DataFrame(_dict_ma_duotou_1)
_df_ma_duotou_1

,book_id,close_up_ratio,continue_days,start_date
0,000001.XSHE,0.775946,1,2005-05-25
1,000001.XSHE,0.775946,2,2005-05-27
2,000001.XSHE,0.775946,6,2005-06-10
3,000001.XSHE,0.775946,3,2005-08-15
4,000001.XSHE,0.775946,4,2005-09-06
5,000001.XSHE,0.775946,6,2005-09-13
6,000001.XSHE,0.775946,14,2005-12-23
7,000001.XSHE,0.775946,3,2006-02-08
8,000001.XSHE,0.775946,9,2006-02-20
9,000001.XSHE,0.775946,10,2006-04-10


In [32]:
print("持续天数中位数{}".format(np.median(_df_ma_duotou_1['continue_days'])))

持续天数中位数7.0


In [33]:
print("上涨中位数{}".format(np.median(_df_ma_duotou_1['close_up_ratio'])*100))

上涨中位数77.59456838021345


In [34]:
## 如下是查看，比如已经连续6天，判断还继续连续下去的几率，这个只是判断到连续60天
# 总的数量
_sum_count=len(_df_ma_duotou_1)
for _days_i in range(1,60):
    # 2个天数，一个是当天的数量，一个是大于当天的数量
    _count_1=len(_df_ma_duotou_1[_df_ma_duotou_1.continue_days==_days_i])
    _count_2=len(_df_ma_duotou_1[_df_ma_duotou_1.continue_days>_days_i])
    _count_3=_count_1+_count_2
    # 几个比率，一个是当天的比率。另一个是持续下去的比率，另一个是到第几天为止，存货的几率。
    _ratio_1=_count_1/_sum_count*100
    _ratio_2=_count_2/_count_3*100
    _ratio_3=_count_2/_sum_count*100
    print("天数：{} 占比：{:.1f}% 继续下去比率{:.1f}% 存活几率：{:.1f}%".format(_days_i,_ratio_1,_ratio_2,_ratio_3))
    

天数：1 占比：9.8% 继续下去比率90.2% 存活几率：90.2%
天数：2 占比：7.9% 继续下去比率91.3% 存活几率：82.3%
天数：3 占比：6.9% 继续下去比率91.6% 存活几率：75.4%
天数：4 占比：6.5% 继续下去比率91.3% 存活几率：68.8%
天数：5 占比：7.9% 继续下去比率88.5% 存活几率：60.9%
天数：6 占比：7.9% 继续下去比率87.1% 存活几率：53.0%
天数：7 占比：7.5% 继续下去比率85.8% 存活几率：45.5%
天数：8 占比：6.7% 继续下去比率85.4% 存活几率：38.9%
天数：9 占比：5.5% 继续下去比率85.9% 存活几率：33.4%
天数：10 占比：4.2% 继续下去比率87.5% 存活几率：29.2%
天数：11 占比：3.8% 继续下去比率87.1% 存活几率：25.4%
天数：12 占比：3.3% 继续下去比率87.1% 存活几率：22.1%
天数：13 占比：3.0% 继续下去比率86.4% 存活几率：19.1%
天数：14 占比：2.7% 继续下去比率86.0% 存活几率：16.5%
天数：15 占比：2.2% 继续下去比率86.9% 存活几率：14.3%
天数：16 占比：1.9% 继续下去比率86.5% 存活几率：12.4%
天数：17 占比：1.7% 继续下去比率86.2% 存活几率：10.6%
天数：18 占比：1.4% 继续下去比率86.4% 存活几率：9.2%
天数：19 占比：1.2% 继续下去比率86.5% 存活几率：7.9%
天数：20 占比：1.1% 继续下去比率86.3% 存活几率：6.9%
天数：21 占比：1.0% 继续下去比率85.6% 存活几率：5.9%
天数：22 占比：0.8% 继续下去比率85.6% 存活几率：5.0%
天数：23 占比：0.7% 继续下去比率86.0% 存活几率：4.3%
天数：24 占比：0.6% 继续下去比率85.6% 存活几率：3.7%
天数：25 占比：0.5% 继续下去比率85.4% 存活几率：3.2%
天数：26 占比：0.5% 继续下去比率85.5% 存活几率：2.7%
天数：27 占比：0.4% 继续下去比率86.7% 存活几率：2.3%
天数：28 占比：0.3% 继续下去比率

In [36]:
print("看看时间统计")
_df_ma_duotou_1["start_date"].value_counts()

看看时间统计


2016-04-06    439
2015-11-25    429
2015-11-09    427
2008-12-22    394
2014-09-03    363
2009-03-23    330
2015-04-27    323
2014-09-04    319
2009-11-05    314
2013-01-18    309
2009-04-15    305
2010-08-19    303
2009-12-04    286
2016-04-05    285
2013-02-01    285
2015-03-02    276
2012-03-13    275
2015-11-06    274
2014-09-24    261
2015-03-18    257
2013-02-18    254
2015-12-23    253
2009-01-16    252
2016-04-15    252
2013-01-31    249
2015-06-05    249
2013-01-15    243
2015-05-14    242
2008-12-19    240
2016-04-07    238
             ... 
2008-09-03      1
2015-07-16      1
2015-07-31      1
2008-08-13      1
2008-07-21      1
2007-01-29      1
2008-07-18      1
2008-10-27      1
2008-10-15      1
2008-04-24      1
2008-11-05      1
2008-09-12      1
2007-06-05      1
2008-04-09      1
2008-06-23      1
2016-02-02      1
2008-08-25      1
2015-07-07      1
2008-03-19      1
2015-09-15      1
2015-08-28      1
2008-10-14      1
2011-05-30      1
2007-03-05      1
2008-04-03

In [47]:
_tmp_1=_df_ma_duotou_1.groupby('start_date')
_tmp_1=_tmp_1.count()
print("每天出现均线多头的股票数量中位数:{}".format(np.median(_tmp_1['book_id'])))
print("每天出现均线多头股票的数量平均数量：{:.1f}".format(sum(_tmp_1['book_id'])/len(_tmp_1['book_id'])))
print("得筛选股票了")

每天出现均线多头的股票数量中位数:38.0
每天出现均线多头股票的数量平均数量：54.0
得筛选股票了
